In [ ]:
import py_entitymatching as em
import pandas as pd
import os
from sdv.tabular import CTGAN
from IPython.display import display, HTML
from utils.utils import *

pd.options.mode.chained_assignment = None  # default='warn''

<h3>0. Prelude.</h3>
This Notebook takes care of the CTGAN traning pipeline. <br>
It takes into consideration if the data format is either DITTO or Magellan. <br>
Thereafter it seperates the data on the matched value to determine what type of data its traning on. <br>

In [ ]:
# If the data is in DITTO format or not.
ditto_format = True

# Data table directory and name.
datasets_dir = r'C:\Users\aleks\Desktop\Master Thesis\Idun\CTGAN\Datasets\er_magellan\Dirty\DBLP-ACM'
name_of_table = "test.txt"

# Model training parameters.
epochs = 1000
batch_total = 1000

# Model directory and name to be saved. 
model_dir = r'C:\Users\aleks\Desktop\Master Thesis\Py_Magellan\Models'
model_name = "Placeholder.pkl"

# If the model should trained on "matched" or "non-matched" examples.
train_on_matched = True

<h3>1. Load dataset for traning.</h3>

In [ ]:
if ditto_format:
    ditto_data_path = datasets_dir + os.sep + name_of_table
    
    with open(ditto_data_path, 'r', encoding='utf-8') as file:
        data = file.read()

    table_A, table_B, truth_table = ditto_reformater(data)
    
    # Conjoin tables together with Truth
    table_A = table_A.add_prefix("ltable_")
    table_B = table_B.add_prefix("rtable_")
    table = pd.concat([table_A, table_B, truth_table], axis=1)

else:
    print("Please perform the Magellan Sampling pipeline before proceeding.") 
    magellan_data_path = datasets_dir + os.sep + name_of_table
    table = pd.read_csv(magellan_data_path)


<h4>2. Train model on data.</h4>
We train our CTGAN model on the loaded table with the given parameters "epochs" and "batch_total". <br>
We keep only "matched" or "unmatched" data, as we seperate the classifications before feeding it to our model. <br>
The table needs an "_id" column as index. <br>

In [ ]:
if train_on_matched:
    table_for_training = table[table['Truth'] > 0]
else:
    table_for_training = table[table['Truth'] < 1]

model = CTGAN(primary_key='_id', epochs=epochs, batch_size=batch_total)
model.fit(table_for_training)
model_save_path = model_dir + os.sep + model_name
model.save(model_save_path)
